In [1]:
%load_ext sql
%sql sqlite:///chinook.db

## Задание 1
Для каждого артиста вывести все жанры, которые есть в его песнях, и для каждого жанра вывести наиболее продолжительную песню этого артиста.

In [69]:
%%sql
SELECT ArtistName, GenreName,TrackName FROM
(
    SELECT ar.Name as ArtistName,g.Name as GenreName, t.Name as TrackName, 
    rank()
        OVER (PARTITION BY ar.Name,g.Name ORDER BY t.milliseconds DESC) rn

    FROM Tracks t
    JOIN Albums al ON al.AlbumId = t.AlbumId
    JOIN Artists ar ON al.ArtistId = ar.ArtistId
    JOIN Genres g ON g.GenreId = t.GenreId
)
WHERE rn = 1

LIMIT 20

 * sqlite:///chinook.db
Done.


ArtistName,GenreName,TrackName
AC/DC,Rock,Overdose
Aaron Copland & London Symphony Orchestra,Classical,Fanfare for the Common Man
Aaron Goldberg,Jazz,OAM's Blues
Academy of St. Martin in the Fields & Sir Neville Marriner,Classical,Fantasia On Greensleeves
Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner,Classical,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525: I. Allegro"
"Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair",Classical,"Requiem, Op.48: 4. Pie Jesu"
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",Classical,"Suite No. 3 in D, BWV 1068: III. Gavotte I & II"
Accept,Rock,Princess of the Dawn
Adrian Leaper & Doreen de Feis,Classical,"Symphony No. 3 Op. 36 for Orchestra and Soprano ""Symfonia Piesni Zalosnych"" \ Lento E Largo - Tranquillissimo"
Aerosmith,Rock,Livin' On The Edge


## Задание 2
Посчитать скользящее среднее выручки по годам.


In [111]:
%%sql

SELECT CustomerId, strftime('%Y', InvoiceDate) as Year,
    max(sum(total)) 
        OVER (PARTITION BY CustomerId ORDER BY InvoiceDate) Curr_Total,

    max(avg(total)) OVER (PARTITION BY CustomerId ORDER BY InvoiceDate ROWS BETWEEN UNBOUNDED PRECEDING AND current row) Average
FROM Invoices i

GROUP BY CustomerId, Year
LIMIT 20

 * sqlite:///chinook.db
Done.


CustomerId,Year,Curr_Total,Average
1,2010,13.879999999999999,4.626666666666666
1,2011,13.879999999999999,4.626666666666666
1,2012,15.84,7.92
1,2013,15.84,8.91
2,2009,24.75,8.25
2,2011,24.75,8.25
2,2012,24.75,8.25
3,2010,26.75,8.916666666666666
3,2012,26.75,8.916666666666666
3,2013,26.75,8.916666666666666


## Задание 3

Для каждого плейлиста вывести трек с наибольшей продолжительностью, трек с наименьшей стоимостью (если у треков одинаковая стоимость, то вывести тот, который весит меньше всех), и жанр данного плейлиста (жанром плейлиста будет называться тот жанр, треков которого больше всех в плейлисте).

In [206]:
%%sql

WITH max_long_track as
(
    SELECT PlaylistId, Name as Longest FROM
    (
        SELECT *,
        row_number()
            OVER(PARTITION BY PlaylistId ORDER BY t.milliseconds DESC) rn
        FROM PlayList_track pl
        JOIN Tracks t ON pl.TrackId = t.TrackId
    )
    WHERE rn = 1
),

min_price_track as
(
    SELECT PlaylistId, Name as Cheapest, UnitPrice FROM
    (
        SELECT *,
        row_number()
            OVER(PARTITION BY PlaylistId ORDER BY UnitPrice, Bytes) rn
        FROM Playlist_track pl
        JOIN Tracks t ON pl.TrackId = t.TrackId
    )
    WHERE rn = 1
),

playlist_genre as
(
    SELECT DISTINCT PlaylistId, GenreName as Playlist_Genre FROM
    (
        SELECT *, g.Name as GenreName,
            count(*)
                OVER(PARTITION BY PlaylistId,t.GenreId) cnt
                
        FROM playlist_track pl
        JOIN tracks t ON t.TrackId = pl.TrackId
        JOIN Genres g ON g.GenreId = t.GenreId
    )
    GROUP BY playlistId

)

SELECT mpt.PlaylistId, Longest, Cheapest, Playlist_Genre FROM 
(
    max_long_track mxt
)
JOIN min_price_track mpt ON mpt.PlaylistId = mxt.PlaylistId
JOIN playlist_genre plg ON mpt.PlaylistId = plg.PlaylistId



 * sqlite:///chinook.db
Done.


PlaylistId,Longest,Cheapest,Playlist_Genre
1,Dazed And Confused,É Uma Partida De Futebol,Rock
3,Occupation / Precipice,LOST Season 4 Trailer,Science Fiction
5,Dazed And Confused,É Uma Partida De Futebol,Rock
8,Dazed And Confused,É Uma Partida De Futebol,Rock
9,"Band Members Discuss Tracks from ""Revelations""","Band Members Discuss Tracks from ""Revelations""",Alternative
10,Occupation / Precipice,LOST Season 4 Trailer,Science Fiction
11,Vai Passar,"Maria, Maria",Latin
12,"Adagio for Strings from the String Quartet, Op. 11","L'orfeo, Act 3, Sinfonia (Orchestra)",Soundtrack
13,"Symphony No. 3 Op. 36 for Orchestra and Soprano ""Symfonia Piesni Zalosnych"" \ Lento E Largo - Tranquillissimo","L'orfeo, Act 3, Sinfonia (Orchestra)",Soundtrack
14,"Symphonie Fantastique, Op. 14: V. Songe d'une nuit du sabbat","Lamentations of Jeremiah, First Set \ Incipit Lamentatio",Classical


## Задание 4

Для каждого жанра вывести треки, которые встречались в большем числе заказов,
сумму, потраченную на данные треки, количество купленных треков и компанию, которая больше всех приобрела данную композицию.

In [2]:
%%sql

WITH track_invoice_count as
(
    SELECT GenreName,TrackId,cnt,Total_sum FROM
    (
        SELECT *,cnt * Totals Total_sum,
            rank()
                OVER(PARTITION BY GenreName ORDER BY cnt DESC) rank
        FROM
        (
            SELECT g.Name as GenreName, ii.TrackId, count(*) cnt, ii.UnitPrice * ii.quantity Totals
            FROM Invoice_items ii
            JOIN Tracks t ON t.TrackId = ii.TrackId
            JOIN Genres g ON g.GenreId = t.Genreid
            GROUP BY ii.TrackId 
            having cnt
        )  
    ) 
    WHERE rank = 1
    
)


SELECT *,
    (
     SELECT c.Company 
     FROM customers c 
     JOIN invoices i ON c.CustomerId = i.CustomerId 
     JOIN invoice_items ii ON i.InvoiceId = ii.InvoiceId 

     WHERE ii.TrackId = track_invoice_count.TrackId AND c.Company is not NULL
     GROUP BY c.Company 
     ORDER BY count(*) DESC 
     LIMIT 1
     ) as top_company

FROM track_invoice_count

LIMIT 30


 * sqlite:///chinook.db
Done.


GenreName,TrackId,cnt,Total_sum,top_company
Alternative,3365,1,0.99,None
Alternative,3369,1,0.99,None
Alternative,3373,1,0.99,None
Alternative,3374,1,0.99,None
Alternative,3378,1,0.99,None
Alternative,3382,1,0.99,None
Alternative,3383,1,0.99,None
Alternative,3387,1,0.99,None
Alternative,3391,1,0.99,None
Alternative,3392,1,0.99,None


## Задание 5
Составить рейтинг клиентов по количеству потраченных ими денег.

In [296]:
%%sql

SELECT *,
  dense_rank()
        OVER(ORDER BY sums DESC) rank
FROM
(
    SELECT c.CustomerId, sum(Total) sums
        
    FROM Customers c
    JOIN Invoices i ON c.CustomerId = i.CustomerId
    GROUP BY i.CustomerId
    having sums
)

 * sqlite:///chinook.db
Done.


CustomerId,sums,rank
6,49.620000000000005,1
26,47.620000000000005,2
57,46.62,3
45,45.62,4
46,45.62,4
28,43.620000000000005,5
24,43.62,6
37,43.62,6
7,42.62,7
25,42.62,7
